In [4]:
import pandas as pd

csv_files = [
    'D:/assignment 2/CSV1.csv',
    'D:/assignment 2/CSV2.csv',
    'D:/assignment 2/CSV3.csv',
    'D:/assignment 2/CSV4.csv'
]

all_texts = []

for file in csv_files:
    try:
        df = pd.read_csv(file)

        print(f"Column names in {file}: {df.columns}")

        text_column = None
        for col in df.columns:
            if 'text' in col.lower():
                text_column = col
                break

        if text_column is not None:
            texts = df[text_column].astype(str)
            all_texts.extend(texts)
        else:
            print(f"Warning: No text column found in {file}")

    except pd.errors.EmptyDataError:
        print(f"Warning: Empty file found - {file}")

if all_texts:
    combined_text = "\n".join(all_texts)
    output_file = 'D:/assignment 2/combined_txt.txt'
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(combined_text)

    print(f'Combined texts saved to {output_file}')
else:
    print('No valid text columns found in the CSV files.')


Column names in D:/assignment 2/CSV1.csv: Index(['Unnamed: 0', 'HADM_ID', 'SHORT-TEXT', 'ICD9_CODE', 'ICD9', 'Label'], dtype='object')
Column names in D:/assignment 2/CSV2.csv: Index(['Unnamed: 0', 'HADM_ID', 'TEXT', 'LABLE', 'entites', 'group'], dtype='object')
Column names in D:/assignment 2/CSV3.csv: Index(['HADM_ID', 'TEXT', 'ICD9_CODE', 'Label'], dtype='object')
Column names in D:/assignment 2/CSV4.csv: Index(['HADM_ID', 'TEXT', 'LABLE'], dtype='object')
Combined texts saved to D:/assignment 2/combined_txt.txt


In [5]:
from collections import Counter
import csv

file_path = 'D:/assignment 2/combined_txt.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    exit()

words = text.split()

word_counts = Counter(words)

top_30_words = word_counts.most_common(30)

for word, count in top_30_words:
    print(f'{word}: {count}')
    
csv_file_path = 'D:/assignment 2/top30_words.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Word', 'Count']) 
    csv_writer.writerows(top_30_words)
print(f'Top 30 words and their counts saved to {csv_file_path}')

and: 2341409
the: 2158529
to: 1931537
of: 1926215
was: 1867807
with: 1366827
a: 1151764
on: 1104052
in: 1000773
for: 976439
is: 775103
mg: 725803
no: 552340
patient: 525955
The: 497153
at: 496295
or: 479400
Tablet: 452050
as: 450188
PO: 408981
(1): 381504
Sig:: 371957
**]: 371169
Name: 364713
were: 347112
he: 346640
his: 346495
her: 329535
left: 326477
had: 321160
Top 30 words and their counts saved to D:/assignment 2/top30_words.csv


In [ ]:
from transformers import AutoTokenizer
from collections import Counter

def count_unique_tokens(text, model_name='bert-base-uncased'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = tokenizer.tokenize(text)
    token_counts = Counter(tokens)
    return token_counts.most_common(30)

text_file_path = 'D:/assignment 2/combined_txt.txt'
model_name = 'bert-base-uncased'

with open(text_file_path, 'r') as file:
    text = file.read()

top_30_tokens = count_unique_tokens(text)
print(top_30_tokens)


In [1]:
import spacy

nlp = spacy.load("en_core_sci_sm")

def extract_entities(text):
    doc = nlp(text)
    entities = {"diseases": [], "drugs": []}
    
    for ent in doc.ents:
        if ent.label_ == "DISEASE":
            entities["diseases"].append(ent.text)
        elif ent.label_ == "DRUG":
            entities["drugs"].append(ent.text)
    
    return entities

file_path = "D:/assignment 2/combined_txt.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

result = extract_entities(text)

print("Diseases:", result["diseases"])
print("Drugs:", result["drugs"])


OSError: [E050] Can't find model 'en_core_sci_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
import spacy

model = BertForTokenClassification.from_pretrained("monologg/biobert_v1.1_pubmed", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")

labels = ["O", "B-Disease", "I-Disease"]


def extract_entities(text):

    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_tensor = torch.tensor([input_ids])

    with torch.no_grad():
        outputs = model(input_tensor)

    predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

    entities = []
    current_entity = None

    for token, label_id in zip(tokens, predictions):
        label = labels[label_id]
        if label.startswith("B-"):
            if current_entity is not None:
                entities.append(current_entity)
            current_entity = {"label": label[2:], "text": [token]}
        elif label.startswith("I-"):
            if current_entity is not None:
                current_entity["text"].append(token)
        else:
            if current_entity is not None:
                entities.append(current_entity)
            current_entity = None

    if current_entity is not None:
        entities.append(current_entity)

    extracted_entities = [{"label": entity["label"], "text": " ".join(entity["text"])} for entity in entities]
    
    return extracted_entities

file_path = "D:/assignment 2/combined_txt.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

result = extract_entities(text)

for entity in result:
    print(f"Label: {entity['label']}, Text: {entity['text']}")


D:\Anaconda\envs\scispacy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the che